In [1]:
import pandas as pd
from google.cloud import bigquery
import pickle
import os

In [2]:
client = bigquery.Client()
merch_data_ref = client.dataset('merch_store', project = 'dummy24571')

In [3]:
def bq2df(sql):
    query = client.query(sql) # API request
    results = query.result()
    return results.to_dataframe()

In [4]:
q = ('SELECT * FROM `dummy24571.merch_store.ga_sessions_*` '
    'LIMIT 100')

In [5]:
# bq2df(q)

In [6]:
merch_data = client.get_dataset(merch_data_ref)

In [7]:
merch_data

Dataset(DatasetReference('dummy24571', 'merch_store'))

In [8]:
# [x.table_id for x in client.list_tables(merch_data)]

In [9]:
merch_full = client.get_table(merch_data.table('ga_sessions_20160801'))

In [10]:
merch_full

Table(TableReference(DatasetReference('dummy24571', 'merch_store'), 'ga_sessions_20160801'))

In [11]:
[command for command in dir(merch_full) if not command.startswith('_')]

['clustering_fields',
 'created',
 'dataset_id',
 'description',
 'encryption_configuration',
 'etag',
 'expires',
 'external_data_configuration',
 'friendly_name',
 'from_api_repr',
 'from_string',
 'full_table_id',
 'labels',
 'location',
 'modified',
 'mview_enable_refresh',
 'mview_last_refresh_time',
 'mview_query',
 'mview_refresh_interval',
 'num_bytes',
 'num_rows',
 'partition_expiration',
 'partitioning_type',
 'path',
 'project',
 'range_partitioning',
 'reference',
 'require_partition_filter',
 'schema',
 'self_link',
 'streaming_buffer',
 'table_id',
 'table_type',
 'time_partitioning',
 'to_api_repr',
 'to_bqstorage',
 'view_query',
 'view_use_legacy_sql']

In [20]:
# merch_full.schema

In [75]:
query = """
  SELECT
      fullVisitorId AS visitor_id,
      visitId AS visit_id,
      CONCAT(fullVisitorId, CAST(visitId AS STRING)) AS unique_session_id,
      date,
      product.v2ProductCategory AS product_category,
      product.v2ProductName AS product_name,
      product.productSKU AS product_sku,
      product.productPrice/1e6 AS product_price,
      product.productQuantity AS product_quantity,
      product.productRevenue/1e6 AS product_revenue,
      totals.totalTransactionRevenue/1e6 AS total_revenue
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
      , UNNEST(hits) AS hits
      , UNNEST(hits.product) AS product
  WHERE
      _TABLE_SUFFIX BETWEEN '20170401' AND '20170430'
      AND geoNetwork.country = 'United States'
      AND productRevenue IS NOT NULL
  ORDER BY
      4 ASC, 10 DESC
"""


In [91]:
data_0 = bq2df(query)

In [93]:
data_0

,visitor_id,visit_id,unique_session_id,date,product_category,product_name,product_sku,product_price,product_quantity,product_revenue,total_revenue
0,6818117303364696962,1491091099,68181173033646969621491091099,20170401,Apparel,Google Women's Fleece Hoodie,GGOEGAQB036014,44.79,1,47.040000,164.17
1,6818117303364696962,1491091099,68181173033646969621491091099,20170401,Apparel,Google Men's Zip Hoodie,GGOEGAFB035814,44.79,1,47.040000,164.17
2,6818117303364696962,1491091099,68181173033646969621491091099,20170401,Apparel,Google Youth Girl Hoodie,GGOEGAYQ068956,35.20,1,37.450000,164.17
3,6818117303364696962,1491091099,68181173033646969621491091099,20170401,Apparel,Google Toddler Hoodie Royal Blue,GGOEGAXC065629,30.39,1,32.640000,164.17
4,2418102200342405484,1491072529,24181022003424054841491072529,20170401,Apparel,Google Onesie Red,GGOEGAWR061148,23.99,1,25.990000,25.99
...,...,...,...,...,...,...,...,...,...,...,...
2772,1206882682891516620,1493605954,12068826828915166201493605954,20170430,Office,Recycled Paper Journal Set,GGOEGOCC017599,3.99,1,4.704285,48.38
2773,4859141843536295311,1493617449,48591418435362953111493617449,20170430,Waze,Waze Mood Original Window Decal,GGOEWFKA083199,1.59,1,2.732857,69.45
2774,4859141843536295311,1493617449,48591418435362953111493617449,20170430,Waze,Waze Mood Ninja Window Decal,GGOEWFKA083299,1.59,1,2.732857,69.45
2775,1206882682891516620,1493605954,12068826828915166201493605954,20170430,Office,Ballpoint Pen Blue,GGOEGOAC021799,0.99,2,2.694285,48.38


In [46]:
query = """
  SELECT
      fullVisitorId AS visitor_id,
      visitId AS visit_id,
      CONCAT(fullVisitorId, CAST(visitId AS STRING)) AS unique_session_id,
      date,
      product.v2ProductCategory AS product_category,
      product.v2ProductName AS product_name,
      product.productSKU AS product_sku,
      product.productPrice/1e6 AS product_price,
      totals.timeOnScreen AS time_spent,
      totals.transactions AS transection,
      product.productQuantity AS product_quantity,
      product.productRevenue/1e6 AS product_revenue,
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
      , UNNEST(hits) AS hits
      , UNNEST(hits.product) AS product
  WHERE
      _TABLE_SUFFIX BETWEEN '20170401' AND '20170531'
      AND geoNetwork.country = 'United States'
      AND productRevenue IS NOT NULL
  ORDER BY
      4 ASC
"""


In [47]:
data_1 = bq2df(query)

In [48]:
data_1

,visitor_id,visit_id,unique_session_id,date,product_category,product_name,product_sku,product_price,time_spent,transection,product_quantity,product_revenue
0,2556948849524550835,1491097844,25569488495245508351491097844,20170401,Apparel,Google Men's Short Sleeve Hero Tee Heather,GGOEGAAJ073415,18.99,NaN,1,1,23.99
1,1618884882553763938,1491090577,16188848825537639381491090577,20170401,Apparel,Google Men's 100% Cotton Short Sleeve Hero Tee...,GGOEGAAB010517,16.99,NaN,1,1,24.99
2,1639589047263376521,1491095245,16395890472633765211491095245,20170401,Apparel,Google Womens 3/4 Sleeve Baseball Raglan Heath...,GGOEGAEA030413,15.99,NaN,1,1,20.49
3,1639589047263376521,1491095245,16395890472633765211491095245,20170401,Apparel,Google Women's Short Sleeve Badge Tee Red Heather,GGOEGAER029213,15.19,NaN,1,1,19.69
4,2418102200342405484,1491072529,24181022003424054841491072529,20170401,Apparel,Google Onesie Red,GGOEGAWR061148,23.99,NaN,1,1,25.99
...,...,...,...,...,...,...,...,...,...,...,...,...
5911,8637158932301658318,1496250783,86371589323016583181496250783,20170531,Gift Cards,Gift Card - $25.00,GGOEGGCX056299,25.00,NaN,15,1,32.00
5912,8637158932301658318,1496250783,86371589323016583181496250783,20170531,Gift Cards,Gift Card - $25.00,GGOEGGCX056299,25.00,NaN,15,1,32.00
5913,8637158932301658318,1496250783,86371589323016583181496250783,20170531,Gift Cards,Gift Card - $25.00,GGOEGGCX056299,25.00,NaN,15,1,32.00
5914,8637158932301658318,1496250783,86371589323016583181496250783,20170531,Gift Cards,Gift Card - $25.00,GGOEGGCX056299,25.00,NaN,15,1,32.00


In [53]:
query = """
  SELECT
      CONCAT(fullVisitorId, CAST(visitId AS STRING)) AS unique_session_id,
      date,
      totals.totalTransactionRevenue/1e6 AS total_revenue
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
      _TABLE_SUFFIX BETWEEN '20170401' AND '20170430'
      AND geoNetwork.country = 'United States'
      AND totals.totalTransactionRevenue IS NOT NULL
"""

In [54]:
data_2 = bq2df(query)

In [55]:
data_2

,unique_session_id,date,total_revenue
0,23031991900246223211493140374,20170425,57.97
1,00322853888042345731493134510,20170425,146.50
2,95898221681013002091493127285,20170425,341.75
3,18544058375709823701493126454,20170425,54.67
4,60066570470165770711493151594,20170425,28.19
...,...,...,...
889,03799108209105408081491329068,20170404,34.49
890,04156086765464325591491340648,20170404,122.98
891,01029675510187575791491370615,20170404,61.93
892,46004361868375183581491361567,20170404,31.98


In [60]:
query = """
  SELECT
      fullVisitorId AS visitor_id,
      visitId AS visit_id,
      date,
      product.v2ProductCategory AS product_category,
      product.v2ProductName AS product_name,
      product.productSKU AS product_sku,
      product.productPrice/1e6 AS product_price,
      product.productQuantity AS product_quantity,
      product.productRevenue/1e6 AS product_revenue,
      totals.totalTransactionRevenue/1e6 AS total_revenue
  FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
      , UNNEST(hits) AS hits
      , UNNEST(hits.product) AS product
  WHERE
      _TABLE_SUFFIX BETWEEN '20170401' AND '20170430'
      AND geoNetwork.country = 'United States'
      AND productRevenue IS NOT NULL
  ORDER BY
      4 ASC, 10 DESC
"""


In [61]:
data_3 = bq2df(query)

In [79]:
data_3.shape

(2777, 10)

In [80]:
data_3.head()

,visitor_id,visit_id,date,product_category,product_name,product_sku,product_price,product_quantity,product_revenue,total_revenue
0,6104963094502152821,1493314922,20170427,${productitem.product.origCatName},Google Sunglasses,GGOEGHGC019799,2.80,13,36.733333,463.54
1,0608915197735218105,1492021292,20170412,${productitem.product.origCatName},Google Doodle Decal,GGOEGFKQ020799,2.99,4,12.498461,363.32
2,0717343852152056082,1492457532,20170417,${productitem.product.origCatName},Recycled Mouse Pad,GGOEGODR017799,1.20,6,7.800000,278.00
3,5263059809967138936,1491587620,20170407,${productitem.product.origCatName},Google Youth Baseball Raglan Heather/Black,GGOEGAYB068056,19.99,2,40.070909,259.52
4,8657427332734176422,1491585605,20170407,${productitem.product.origCatName},Recycled Mouse Pad,GGOEGODR017799,1.20,35,44.000000,241.30


In [ ]:
df = pd.read_csv(('../data/raw/raw.csv'))

In [ ]:
df.head()

In [3]:
import platform

In [4]:
platform.python_version()

'3.6.5'

In [1]:
from google.cloud import bigquery